# Introduce an Agent workflow

# XX Agent before LMCache

# XX Agent with LMCache

# Improvement breakdown
Show how LMCache saves processing time for each step and eventually improved the overall processing speed.